## Predicting breast cancer from digitized images of breast mass

by Tiffany A. Timbers, Joel Ostblom & Melissa Lee
2023/11/09

In [1]:
import pandas as pd
from myst_nb import glue
import pickle
#from sklearn import set_config

In [2]:
test_scores_df = pd.read_csv("../results/tables/test_scores.csv").round(2)
glue("accuracy", test_scores_df['accuracy'].values[0], display=False)
glue("f2", test_scores_df['F2 score (beta = 2)'].values[0], display=False)
test_scores_df = test_scores_df.style.format().hide()
glue("test_scores_df", test_scores_df, display=False)


In [3]:
confusion_df=pd.read_csv("../results/tables/confusion_matrix.csv", index_col='class')
confusion_df.rename(columns={'Benign':'Predicted: Benign'}, inplace=True)
confusion_df.index.names = ['Actual label:']
glue("total", confusion_df.sum(axis=1).sum(), display=False)
glue("pred_correct", confusion_df['Predicted: Benign'].values[0] + confusion_df['Malignant'].values[1], display=False)
glue("false_neg", confusion_df['Predicted: Benign'].values[1], display=False)
glue("confusion_df", confusion_df, display=False)

In [4]:
with open('../results/models/cancer_pipeline.pickle', 'rb') as f:
    cancer_fit = pickle.load(f)
glue("best_k", cancer_fit.best_params_['kneighborsclassifier__n_neighbors'], display=False) 


## Summary

Here we attempt to build a classification model using the k-nearest neighbours algorithm which can use breast cancer tumour image measurements to predict whether a newly discovered breast cancer tumour is benign (i.e., is not harmful and does not require treatment) or malignant (i.e., is harmful and requires treatment intervention). Our final classifier performed fairly well on an unseen test data set, with the F2 score, where beta = 2, of {glue:text}`f2` and an overall accuracy calculated to be {glue:text}`accuracy`. On the {glue:text}`total` test data cases, it correctly predicted {glue:text}`pred_correct`. 

However it incorrectly predicted {glue:text}`false_neg` cases, and importantly these cases were false negatives; predicting that a tumour is benign when in fact it is malignant. These kind of incorrect predictions could have a severly negative impact on a patients health outcome, thus we recommend continuing study to improve this prediction model before it is put into production in the clinic.

## Introduction

Women have a 12.1% lifetime probability of developing breast cancer, and although cancer treatment has improved over the last 30 years, the projected death rate for women's breast cancer is 22.4 deaths per 100,000 in 2019 {cite}`ccsac`. Early detection has been shown to improve outcomes {cite}`ccsac`, and thus methods, assays and technologies that help to improve diagnosis may be beneficial for improving outcomes further. 

Here we ask if we can use a machine learning algorithm to predict whether a newly discovered tumour is benign or malignant given tumour image measurements. Answering this question is important because traditional methods for tumour diagnosis are quite subjective and can depend on the diagnosing physicians skill as well as experience {cite}`Streetetal`. Furthermore, benign tumours are not normally dangerous; the cells stay in the same place and the tumour stops growing before it gets very large. By contrast, in malignant tumours, the cells invade the surrounding tissue and spread into nearby organs where they can cause serious damage. Thus, if a machine learning algorithm can accurately and effectively predict whether a newly discovered tumour benign or malignant given tumour image measurements this could lead to less subjective, and more scalable breast cancer tumour diagnosis which could contribute to better patient outcomes.

## Methods

### Data
The data set used in this project is of digitized breast cancer image features created by Dr. William H. Wolberg, W. Nick Street, and Olvi L. Mangasarian at the University of Wisconsin, Madison {cite}`Streetetal`.  It was sourced from the UCI Machine Learning Repository {cite}`Streetetal` and can be found [here](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)), specifically [this file](http://mlr.cs.umass.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data). Each row in the data set represents summary statistics from measurements of an image of a tumour sample, including the diagnosis (benign or malignant) and several other measurements (e.g., nucleus texture, perimeter, area, etc.). Diagnosis for each image was conducted by physicians. 

### Analysis
The k-nearest neighbors (k-nn) algorithm was used to build a classification model to predict whether a tumour mass was benign or malignant (found in the class column of the data set). All variables included in the original data set, with the exception of the standard error of fractal dimension, smoothness, symmetry and texture were used to fit the model. Data was split with 70% being partitioned into the training set and 30% being partitioned into the test set. The hyperparameter $K$ was chosen using 30-fold cross validation with the F2 score as the classification metric. Beta was chosen to be set to 2 for the F2 score to increase the weight on recall during fitting because the application is cancer screening and false negatives are very undesirable in such an application. All variables were standardized just prior to model fitting. The Python programming language {cite}`Python` and the following Python packages were used to perform the analysis: requests {cite}`requests`, zipfile {cite}`Python`, numpy {cite}`numpy`, Pandas {cite}`mckinney-proc-scipy-2010`, {cite}`altair`, scikit-learn {cite}`scikit-learn`. The code used to perform the analysis and create this report can be found here: https://github.com/ttimbers/breast_cancer_predictor_py.


## Results & Discussion

To look at whether each of the predictors might be useful to predict the tumour class, we plotted the distributions of each predictor from the training data set and coloured the distribution by class (benign: blue and malignant: orange, {numref}`Figure {number} <feature_densities_by_class>`). In doing this we see that class distributions for all of the mean and max predictors for all the measurements overlap somewhat, but do show quite a difference in their centres and spreads. This is less so for the standard error (se) predictors. In particular, the standard errors of fractal dimension, smoothness, symmetry and texture look very similar in both the distribution centre and spread. Thus, we choose to omit these from our model.

```{figure} ../results/figures/feature_densities_by_class.png
---
width: 800px
name: feature_densities_by_class
---
Comparison of the empirical distributions of training data predictors between benign and malignant tumour masses.
```

We chose to use a simple classification model using the k-nearest neighbours algorithm. To find the model that best predicted whether a tumour was benign or malignant, we performed 30-fold cross validation using F2 score (beta = 2) as our metric of model prediction performance to select K (number of nearest neighbours). We observed that the optimal K was {glue:text}`best_k` ({numref}`Figure {number} <cancer_choose_k>`).

```{figure} ../results/figures/cancer_choose_k.png
---
width: 600px
name: cancer_choose_k
---
Results from 30-fold cross validation to choose K. F2 score (with beta = 2) was used as the classification metric as K was varied.
```

```{glue:figure} test_scores_df
:figwidth: 400px
:name: "test_scores_df"

Accuracy and F2 (beta = 2) score from best model predicted on the test data.
```

Our prediction model performed quite well on test data, with a final overall accuracy of {glue:text}`accuracy` and F2 (beta = 2) score of {glue:text}`f2`. Other indicators that our model performed well come from the confusion matrix, where it only made {glue:text}`false_neg` mistakes. However all {glue:text}`false_neg` mistakes were predicting a malignant tumour as benign, given the implications this has for patients health, this model is not good enough to yet implement in the clinic ({numref}`Figure {number} <confusion_df>`).

```{glue:figure} confusion_df
:figwidth: 650px
:name: "confusion_df"

Confusion matrix of model performance on test data.
```

To further improve this model in future with hopes of arriving one that could be used in the clinic, there are several things we can suggest. First, we could look closely at the {glue:text}`false_neg` misclassified observations and compare them to several observations that were classified correctly (from both classes). The goal of this would be to see which feature(s) may be driving the misclassification and explore whether any feature engineering could be used to help the model better predict on observations that it currently is making mistakes on. Additionally, we would try seeing whether we can get improved predictions using other classifiers. One classifier we might try is random forest forest because it automatically allows for feature interaction, where k-nn does not. Finally, we also might improve the usability of the model in the clinic if we output and report the probability estimates for predictions. If we cannot prevent misclassifications through the approaches suggested above, at least reporting a probability estimates for predictions would allow the clinician to know how confident the model was in its prediction. Thus the clinician may then have the ability to perform additional diagnostic assays if the probability estimates for prediction of a given tumour class is not very high.

## References

```{bibliography}
```